In [1]:
import sys 
sys.path.append("/home/karen/Documents/GitHub/Muscle_wasting/network")
sys.path.append("/home/karen/Documents/GitHub/Muscle_wasting/database_analysis")
sys.path.append("/home/karen/Documents/GitHub/Muscle_wasting/venv/lib/python3.10/site-packages")
sys.path.append("/home/karen/Documents/GitHub/Muscle_wasting/venv/lib/python3.10/site-packages/networkx")

In [2]:
cd /home/karen/Documents/GitHub/Muscle_wasting

/home/karen/Documents/GitHub/Muscle_wasting


# Analyzing the relevance of the mirnas

Now we saw with the filtered and whole network
Let's get the mirnas from the relevant PR score and then evaluate them on the whole network

In [3]:
import walking_network as wn

In [4]:
import networkx as nx

In [5]:
import pandas as pd
import numpy as np

We are going to load a graph with the mirnas and with the cut of pagerank to only get relevant genes on the network, however we should do it for the full gone, since genes belonging to pathways may be discarted on this PR evaluation. 

In [6]:
whole_network = nx.read_gpickle(f"network/Networks_pkl/pathway_n_tf_mirnas__tf_network_cutoff_0.pkl")

In [7]:
pr_network = nx.read_gpickle(f"network/Networks_pkl/pathway_n_tf_mirnas__tf_network_cutoff_0.5.pkl")

In [8]:
steps=5

In [9]:
import matplotlib.pyplot as plt
#nx.draw_networkx(network)
#plt.show()

In [10]:
miR_nodes = [node for node in pr_network.nodes if node.startswith('hsa-miR')]
len(miR_nodes)

4

In [11]:
mirPaths={}
for mir in miR_nodes:
    p = wn.get_pathways(graph=whole_network, mirna=mir, n_distance=steps, sample_size=10)
    unique_set = set(tuple(lst) for lst in p)
    
    # Convert back to a list of lists
    unique_list = [list(tpl) for tpl in unique_set]
    mirPaths[mir]=unique_list


In [12]:
p

[['hsa-miR-152'],
 ['hsa-miR-152'],
 ['hsa-miR-152'],
 ['hsa-miR-152'],
 ['hsa-miR-152'],
 ['hsa-miR-152'],
 ['hsa-miR-152'],
 ['hsa-miR-152'],
 ['hsa-miR-152'],
 ['hsa-miR-152']]

In [13]:
mirPaths['hsa-miR-152']

[['hsa-miR-152']]

In [14]:
mirInfluence={}
for mir, path in mirPaths.items():
    influence = wn.get_influence(whole_network, path)
    mirInfluence[mir]=influence
#mirInfluence

In [15]:
mir_pathway_influence = {}
for mir, influence_data in  mirInfluence.items():
    pi = wn.evaluate_pathway_influence(influence_data)
    mir_pathway_influence[mir]=pi

mir_de_influence = {}
for mir, influence_data in  mirInfluence.items():
    pi = wn.evaluate_de_influence(influence_data)
    mir_de_influence[mir]=pi


In [16]:
mir_pathway_influence_df = pd.DataFrame(mir_pathway_influence).T
#mir_pathway_influence_df = mir_pathway_influence_df.drop(columns=["Different_pathways", "Total"])
mir_pathway_influence_df['participation'] = mir_pathway_influence_df.drop(columns=["Different_pathways", "Total"]).sum(axis=1)
mir_pathway_influence_df

,ATP,MITOCHONDRI,RESPIRAT,METABOLI,OXIDATIVE_PHOSPHORYLATION,NONALCOHOLIC_FATTY_LIVER,MUSCLE,ELECTRON,Different_pathways,Total,participation
hsa-miR-21-5p,2,5,4,4,1,1,1,3,21,200,21
hsa-miR-93-5p,0,1,0,0,0,0,0,0,2,16,1
hsa-miR-375,0,0,0,0,0,0,0,0,0,0,0
hsa-miR-152,0,0,0,0,0,0,0,0,0,0,0


In [17]:
mean_influence  = mir_pathway_influence_df.loc[:, 'MITOCHONDRI'].mean()
mean_influence

1.5

In [18]:
mitocondrial_df = mir_pathway_influence_df[mir_pathway_influence_df['MITOCHONDRI']>mean_influence].sort_values('MITOCHONDRI', ascending=False)
mitocondrial_df['Rate']=mir_pathway_influence_df['MITOCHONDRI']/mir_pathway_influence_df['participation']


In [19]:
mitocondrial_df[mitocondrial_df['Different_pathways']<10].sort_values('MUSCLE',ascending=False)

,ATP,MITOCHONDRI,RESPIRAT,METABOLI,OXIDATIVE_PHOSPHORYLATION,NONALCOHOLIC_FATTY_LIVER,MUSCLE,ELECTRON,Different_pathways,Total,participation,Rate


In [20]:
mir_de_influence_df = pd.DataFrame(mir_de_influence).T


In [21]:
mir_de_influence_df[mir_de_influence_df['yo']>0]

,m_l,m_s,yo,ym,mo
hsa-miR-21-5p,0,0,9,3,1
hsa-miR-93-5p,0,0,2,0,0
hsa-miR-375,0,0,1,0,0


In [22]:
mir_de_influence_df[mir_de_influence_df['mo']>0]

,m_l,m_s,yo,ym,mo
hsa-miR-21-5p,0,0,9,3,1


In [23]:
mirs_all_infliences = mir_de_influence_df[(mir_de_influence_df['ym']>0)|(mir_de_influence_df['mo']>0)|(mir_de_influence_df['yo']>0)]
mirs_all_infliences

,m_l,m_s,yo,ym,mo
hsa-miR-21-5p,0,0,9,3,1
hsa-miR-93-5p,0,0,2,0,0
hsa-miR-375,0,0,1,0,0


In [24]:
sel_mirs_list = list(mirs_all_infliences.index)
len(sel_mirs_list)

3

In [25]:
mir_pathway_influence_df_filteres = mir_pathway_influence_df[mir_pathway_influence_df.index.isin(sel_mirs_list)]

In [26]:
mir_pathway_influence_df_filteres#[mir_pathway_influence_df_filteres['MUSCLE']>0].sort_values('MUSCLE')

,ATP,MITOCHONDRI,RESPIRAT,METABOLI,OXIDATIVE_PHOSPHORYLATION,NONALCOHOLIC_FATTY_LIVER,MUSCLE,ELECTRON,Different_pathways,Total,participation
hsa-miR-21-5p,2,5,4,4,1,1,1,3,21,200,21
hsa-miR-93-5p,0,1,0,0,0,0,0,0,2,16,1
hsa-miR-375,0,0,0,0,0,0,0,0,0,0,0


In [27]:
mirPaths['hsa-miR-21-5p']

[['hsa-miR-21-5p', 'PPARA', 'UCP3'],
 ['hsa-miR-21-5p', 'TPM1'],
 ['hsa-miR-21-5p', 'MSH2'],
 ['hsa-miR-21-5p', 'PPARA', 'SLC10A2'],
 ['hsa-miR-21-5p', 'IL1B'],
 ['hsa-miR-21-5p', 'DNM1L'],
 ['hsa-miR-21-5p', 'VHL'],
 ['hsa-miR-21-5p', 'EGLN1'],
 ['hsa-miR-21-5p', 'LRRFIP1', 'TFAP2A', 'CYP11A1']]

In [28]:
mirPaths['hsa-miR-93-5p']

[['hsa-miR-93-5p', 'FOXO3'], ['hsa-miR-93-5p', 'VEGFA']]